In [55]:
from EEGModels import EEGNet
import scipy
import sys
import os
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import LeaveOneOut


In [56]:
folder_path_dance = '../condition_data/dance/raw/exec/'
folder_path_walk = '../condition_data/walk/raw/exec/'

file_list_dance = [os.path.join(folder_path_dance, file) for file in os.listdir(folder_path_dance) if file.startswith('P')]
file_list_walk = [os.path.join(folder_path_walk, file) for file in os.listdir(folder_path_walk) if file.startswith('P')]

file_list_dance.sort()
file_list_walk.sort()

# create a dictionary that maps indices to file names
file_dict_dance = {index: file_name.split("/")[-1] for index, file_name in enumerate(file_list_dance)}
file_dict_walk = {index: file_name.split("/")[-1] for index, file_name in enumerate(file_list_walk)}

X_dance = []
Y_dance = []
X_walk = []
Y_walk = []

for file in file_list_dance:
    dataset = scipy.io.loadmat(file)
    x = dataset['data']
    y = dataset['labels']

    if(y[0] < 4):
        sys.exit("Incorrect label found!")

    y -= 3
    y = np_utils.to_categorical(y-1)
    print(x.shape, y.shape)


    X_dance.append(x)
    Y_dance.append(y)

for file in file_list_walk:
    dataset = scipy.io.loadmat(file)
    x = dataset['data']
    y = dataset['labels']

    if(y[0] < 4):
        sys.exit("Incorrect label found!")

    y -= 3
    y = np_utils.to_categorical(y-1)
    print(x.shape, y.shape)

    X_walk.append(x)
    Y_walk.append(y)


{0: 'P_1.mat', 1: 'P_10.mat', 2: 'P_2.mat', 3: 'P_3.mat', 4: 'P_4.mat', 5: 'P_5.mat', 6: 'P_9.mat'}
['../condition_data/dance/raw/exec/P_1.mat', '../condition_data/dance/raw/exec/P_10.mat', '../condition_data/dance/raw/exec/P_2.mat', '../condition_data/dance/raw/exec/P_3.mat', '../condition_data/dance/raw/exec/P_4.mat', '../condition_data/dance/raw/exec/P_5.mat', '../condition_data/dance/raw/exec/P_9.mat']
['../condition_data/walk/raw/exec/P_1.mat', '../condition_data/walk/raw/exec/P_10.mat', '../condition_data/walk/raw/exec/P_2.mat', '../condition_data/walk/raw/exec/P_3.mat', '../condition_data/walk/raw/exec/P_4.mat', '../condition_data/walk/raw/exec/P_5.mat', '../condition_data/walk/raw/exec/P_9.mat']
(79, 14, 1000) (79, 3)
(56, 14, 1000) (56, 3)
(78, 14, 1000) (78, 3)
(80, 14, 1000) (80, 3)
(116, 14, 1000) (116, 3)
(85, 14, 1000) (85, 3)
(89, 14, 1000) (89, 3)
(84, 14, 1000) (84, 3)
(55, 14, 1000) (55, 3)
(74, 14, 1000) (74, 3)
(84, 14, 1000) (84, 3)
(86, 14, 1000) (86, 3)
(87, 14, 

In [57]:
f = open('../condition_results/data_sizes.txt', 'a')
f.write('P_Dance_Exec: ' + str(sum(x.shape[0] for x in X_dance)) + '\n')
f.write('P_Walk_Exec: ' + str(sum(x.shape[0] for x in X_walk)) + '\n')
f.close()

In [58]:
kernels, chans, samples = 1, 14, 1000

In [67]:
f = open('../condition_results/P_exec.txt', 'a')
f.write('-----------------DANCE-----------------\n')
f.close()

dance_accuracies = []

dance_loo = LeaveOneOut()

for train_index, test_index in dance_loo.split(X_dance):
    X_train = np.concatenate([X_dance[i] for i in train_index])
    Y_train = np.concatenate([Y_dance[i] for i in train_index])

    X_test = np.concatenate([X_dance[i] for i in test_index])
    Y_test = np.concatenate([Y_dance[i] for i in test_index])
    
    print("ITERATION NUMBER: " + str(test_index[0]))

    # convert data to NHWC (trials, channels, samples, kernels) format.
    X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)
    X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)

    # define EEGNet model
    model = EEGNet(nb_classes=3, Chans=chans, Samples=samples, dropoutRate=0.5, 
                   kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

    # compile the model and set the optimizers
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

    # count number of parameters in the model
    numParams    = model.count_params() 

    # set initial class weights
    class_weights = {0: 1., 1: 1., 2: 1.}
    
    # fit the model
    fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = 50, verbose = 2, 
                             class_weight=class_weights)
    
    # make prediction on test set.
    probs = model.predict(X_test)
    preds = probs.argmax(axis = -1)
    acc = balanced_accuracy_score(Y_test.argmax(axis = -1), preds)
    dance_accuracies.append(acc)

    f = open('../condition_results/P_exec.txt', 'a')
    f.write(file_dict_dance[test_index[0]] + ': ' + str(acc) + '\n')
    f.close()

(504, 14, 1000) (504, 3)
(527, 14, 1000) (527, 3)
(505, 14, 1000) (505, 3)
(503, 14, 1000) (503, 3)
(467, 14, 1000) (467, 3)
(498, 14, 1000) (498, 3)
(494, 14, 1000) (494, 3)


In [60]:
f = open('../condition_results/P_exec.txt', 'a')
f.write('-----------------WALK-----------------\n')
f.close()

walk_accuracies = []

walk_loo = LeaveOneOut()

for train_index, test_index in walk_loo.split(X_walk):
    X_train = np.concatenate([X_walk[i] for i in train_index])
    Y_train = np.concatenate([Y_walk[i] for i in train_index])

    X_test = np.concatenate([X_walk[i] for i in test_index])
    Y_test = np.concatenate([Y_walk[i] for i in test_index])
    
    print("ITERATION NUMBER: " + str(test_index[0]))

    # convert data to NHWC (trials, channels, samples, kernels) format.
    X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)
    X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)

    # define EEGNet model
    model = EEGNet(nb_classes=3, Chans=chans, Samples=samples, dropoutRate=0.5, 
                   kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

    # compile the model and set the optimizers
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

    # count number of parameters in the model
    numParams    = model.count_params() 

    # set initial class weights
    class_weights = {0: 1., 1: 1., 2: 1.}
    
    # fit the model
    fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = 50, verbose = 2, 
                             class_weight=class_weights)
    
    # make prediction on test set.
    probs = model.predict(X_test)
    preds = probs.argmax(axis = -1)
    acc = balanced_accuracy_score(Y_test.argmax(axis = -1), preds)
    walk_accuracies.append(acc)

    f = open('../condition_results/P_exec.txt', 'a')
    f.write(file_dict_walk[test_index[0]] + ': ' + str(acc) + '\n')
    f.close()

In [61]:
f = open('../condition_results/P_exec.txt', 'a')

f.write('\n\nP_Dance_accuracy: ' + str(np.mean(dance_accuracies)) + '\n')
f.write('P_Dance_std: ' + str(np.std(dance_accuracies)) + '\n')

f.write('P_Walk_accuracy: ' + str(np.mean(walk_accuracies)) + '\n')
f.write('P_Walk_std: ' + str(np.std(walk_accuracies)) + '\n')

f.close()